In [2]:
import os
import pandas as pd
from tempfile import TemporaryDirectory
from recommenders.datasets.mind import download_mind
from recommenders.datasets.download_utils import unzip_file
from models.als_test import ALSRecommender

In [3]:
# Create tempdir, download and unzip datasets
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_zip, valid_zip = download_mind(size="small", dest_path=data_path)
unzip_file(train_zip, os.path.join(data_path, 'train'), clean_zip_file=False)
unzip_file(valid_zip, os.path.join(data_path, 'valid'), clean_zip_file=False)

# Path to behaviors file
train_behaviors_path = os.path.join(data_path, "train", "behaviors.tsv")
valid_behaviors_path = os.path.join(data_path, "valid", "behaviors.tsv")

100%|██████████| 51.8k/51.8k [00:04<00:00, 10.6kKB/s]
100%|██████████| 30.2k/30.2k [00:03<00:00, 8.88kKB/s]


In [4]:
# Load the behaviors.tsv file into a DataFrame
behaviors_cols = ["impression_id", "user_id", "timestamp", "history", "impressions"]
train_behaviors_df = pd.read_csv(train_behaviors_path, sep='\t', header=None, names=behaviors_cols)
valid_behaviors_df = pd.read_csv(valid_behaviors_path, sep='\t', header=None, names=behaviors_cols)

als_recommender = ALSRecommender(train_behaviors_df, num_factors=20, alpha=40, reg=0.1, num_iters=10)
als_recommender.train()
# For example, generate recommendations for the first user in the dataset:
sample_user = train_behaviors_df.iloc[0]["user_id"]
print("Recommendations for user {}:".format(sample_user))
print(als_recommender.recommend(sample_user, N=5))

Iteration 1/10 completed.
Iteration 2/10 completed.


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the validation set using NDCG@5.
avg_ndcg = als_recommender.evaluate(valid_behaviors_df, K=5)
print("Average NDCG@5 on validation set:", avg_ndcg)